# Justin Butler
## AAE 537: HW 4

This assignment was created in Jupyter Notebook and formatted via $\LaTeX$

In [7]:
%%javascript
// Making sure the outputs display correctly
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [115]:
# Importing required packages for this homework
import numpy as np
import matplotlib.pyplot as plt
#from IPython.display import HTML, display
#import tabulate
%matplotlib inline

def rounds(item):
    ret = float('%.3f'% (item * 1000/1000))
    return(ret)

### i)
For the first part of this assignment, I will be labeling out how to find the stagnation temperature at each station in the engine as well as the fuel/air ratios in the burner and afterburner. We will assume that q, $\beta$, CPR, $C_p$, $\Delta H_B$ (of the fuel), $\dot{m}_0$, T$_{t4}$, and T$_{t7}$ are all known terms. We will also assume we know the efficiencies of the engine, that is, we know $\eta_c, \eta_b, \eta_t, \eta_{ab},$ and $\eta_n$.

#### Inlet
For the inlet, we will use the use the atmospheric conditions at a given altitude as given by the atmospheric model NASA provides at https://www.grc.nasa.gov/WWW/k-12/airplane/atmosmet.html. 

From static pressure we can also find the mach. Assuming $\gamma=1.33$ (or that $\gamma$ is at least a known value) and assuming the ambient air acts like an ideal gas we know:
$$M_0 = \sqrt{\frac{2\cdot q}{\gamma\cdot P_{s0}}}$$

Now with static temperature and mach, we find total temperature at the inlet as:
$$T_{t0} = T_{s0} \left(1 + \frac{\gamma-1}{2}M_0^2\right)$$
From this, we will assume an adiabatic intake - thus $T_{t0} = T_{t2}$. Next, by using the inlet recovery per Mil Std 5008B we can define the piecewise function for pressure ratio as:
$\frac{P_{t2}}{P_{t0}} = \begin{cases}
  1 & \text{for }M_0 <  1\\
  1 - 0.075(M_0-1)^{1.35} & \text{for } 1< M_0 < 5\\
  \frac{800}{M_0^4+935} & \text{for } M_0 > 5 
\end{cases}$

Since stagnation pressure can be calculated as the sum of static and dynamic pressures, we find $P_{t0} = P_{s0} + q$ where $P_{s0}$ is the pressure found from the atmospheric model. From this we can find the total pressure at station 2 from $P_{t2} = P_{t0} \cdot \left(P_{t2}/P_{t0}\right)$.
#### Compressor

For the compressor we can find the stagnation temperature and stagnation pressure for the air by:
$$T_{t3} = T_{t2}\left[1+\frac{1}{\eta_c}\cdot\left(CPR^{(\gamma-1)/\gamma}-1\right)\right]$$
$$P_{t3} = P_{t2}\cdot CPR$$

We will also want to find the massflow running through the core. Since we know $\beta$ and the inlet massflow $\dot{m}$, the massflow going through the core would be:
$$ \dot{m}_c = \frac{\dot{m}}{\beta+1}$$
#### Burner

Moving onto the burner, $T_{t4}$ is a given value and we will assume that $P_{t4} = P_{t3}$. We can find the fuel to air ratio by:
$$f = \frac{T_{t4} - T_{t3}}{\Delta H_B}\cdot C_p$$
#### Turbine

Station 5, within the figure, looks like it occurs before the re-mixing of the core and bypass streams. Thus, we will assume that the massflow out of the core at stage 5 is $\dot{m}_c + \dot{m}_f$. With this value, we will use powerbalance between the compressor and the turbine to determine the total temperature at the end of the turbine. Indeed:
$$T_{t5} = T_{t4} - \frac{\dot{m}_c\cdot C_p\cdot(T_{t3}-T_{t2})}{(\dot{m}_c+\dot{m}_f)\cdot C_p}$$
And from this, we can use isentropic relations to find:
$$P_{t5} = P_{t4}\left(\frac{T_{t5}}{T_{t4}}\right)^\frac{\gamma}{(\gamma-1)}$$
#### Mixer

Looking at the figure, by the time we reach station 6 the bypass flow has rejoined. Thus, the massflow now includes the bypass flow which we will need to account for. For this, we will perform weighted averages (thus assuming a constant area mixer).
$$T_{t6} = \frac{\dot{m}_b}{\dot{m}_c+\dot{m}_f+\dot{m}_b}\cdot T_{t2} + \frac{\dot{m}_c + \dot{m}_f} {\dot{m}_c+\dot{m}_f+\dot{m}_b} \cdot T_{t5}$$
$$P_{t6} = \frac{\dot{m}_b}{\dot{m}_c+\dot{m}_f+\dot{m}_b}\cdot P_{t2} + \frac{\dot{m}_c + \dot{m}_f} {\dot{m}_c+\dot{m}_f+\dot{m}_b} \cdot P_{t5}$$
#### Afterburner

At the afterburner, $T_{t7}$ is a given value and we will again assume that $P_{t7}=P_{t6}$. Thus, the fuel to air ratio is:
$$f_{ab} = \frac{1-T_{t5}}{(\Delta H_B/C_p) - T_{t7}}$$

#### Nozzle

A perfectly expanded nozzle means that $P_9 = P_0$ and that $P_{t9} = P_{t7}$. We will also assume this is an adiabatic process and thus $T_{t9} = T_{t7}$. We can then find the velocity at the exit by:
$$V_9 = \sqrt{2C_pT_{t9}\eta_n\left(1-\left(\frac{P_9}{P_{t9}}\right)^\frac{\gamma-1}{\gamma}\right)}$$


### ii)
The following is the routine that solves for the engine conditions and specific thrust and fuel consumption at a given flight condition and known temperature limits.

In [143]:
# Atmospheric model created using the NASA equation found at:
# https://www.grc.nasa.gov/WWW/k-12/airplane/atmosmet.html
class atmosphere:
    def __init__(self, val, valGiven = 0,units = "SI"):
        #Convert from US to SI
        if units != "SI" and valGiven == 0:
            val = val / 3.281
        if units != "SI" and valGiven == 1:
            val = val * 0.04788
        #0 implies the given value is an altitude
        #1 implies the given value is a pressure
        if valGiven == 0:
            self.h = val
        elif valGiven == 1:
            self.P = val
        else:
            print("Not a valid 'valGiven' parameter.")
    def hCalc(self):
        if self.h < 11000:
            self.T = 15.04 - 0.00649*self.h
            self.P = 101.29 * ((self.T + 273.1)/288.08)**(5.256)
        elif self.h < 25000:
            self.T = -56.46
            self.P = 22.65 * np.exp(1.73 - 0.000157*self.h)
        elif self.h > 24999:
            self.T = -131.21 + 0.00299*self.h
            self.P = 2.488 * ((self.T + 273.1)/216.6)**(-11.388)
        self.rho = self.P / (0.2869 * (self.T + 273.1))
    def PCalc(self):
        if self.P > 22.632:
            self.T = (288.08*(self.P/101.29)**(1/5.256))-273.1
            self.h = (self.T - 15.04)/(-0.00649)
        elif self.P > 0.1113586:
            self.T = -56.46
            self.h = (1.73 - np.log((self.P/22.65)))/(0.000157)
        else:
            self.T = (216.6*(self.P/2.488)**(1/(-11.388)))-273.1
            self.h = (self.T + 131.21)/0.00299
        self.rho = self.P / (0.2869 * (self.T + 273.1))
# ISENTROPIC RELATIONS
def totalPres(Ps, M,gamma=1.4):
    total = Ps * ((1+((gamma-1)/2)*M**2)**(gamma/(gamma-1)))
    return(total)
def statPres(Pt, M, gamma=1.4):
    static = Pt / ((1+((gamma-1)/2)*M**2)**(gamma/(gamma-1)))
    return(static)
def totalTemp(Ts, M, gamma=1.4):
    total = Ts * ((1+((gamma-1)/2)*M**2))
    return(total)
def statTemp(Tt, M, gamma=1.4):
    static = Tt / ((1+((gamma-1)/2)*M**2))
    return(static)        
###################################################
# Engine class constructed from the above analysis#
###################################################
class engine:
    def help():
        print("Engine Class Help File:")
        print("The engine class takes as input several parameters pertaining to a \n"
             "gas turbine engine and calculates information from those starting parameters.\n"
             "The required inputs are in the following order:\n"
              "1. Altitude\n"
              "2. dynamic pressure\n"
              "3. Flow Bypass Ratio \n"
              "4. Compressor Pressure Ratio \n"
              "5. ratio of specific heats\n"
              "6. Total temperature at end of combustor\n"
              "7. Total temperature at exit of the afterburner\n"
              "8. A 5 element list containing the efficiences (c,b,t,ab,n)\n"
              "9. Heating value of the fuel\n"
              "10. Mass flow through the inlet"
             )
        print("\nThese values are set in the class with default values for this problem.\n"
              "Alternatively, one can run with 'manualEntry = True' to be able to enter\n"
              "these values manually when the object is created.\n"
            "Running the 'runAnalysis()' command will generate all of the desired values. \n"
            "These include arrays of temperatures,pressures, mach, areas, and values for the\n"
              "mass flows throughout. There will also be values calculated for the specific thrust\n"
              "and fuel consumption.\n")
        print("For errors in this code, you can contact Justin on Github as user BananaBonanza")
    def __init__(self,manualEntry = False,
                altitude = 0, q = 1500, BPR = 0,
                 CPR = 20, gamma = 1.33, Cp = 0.3,
                 Tt4 = 3000, Tt7 = 3700, nc = 0.88,
                 nb = 0.95, nt = 0.9, nab = 0.9,
                 nn = 0.9,Hb = 18500, mDot = 500
                ):
        if manualEntry:
            inputTerms = ["altitude", "dynamic pressure", "Flow Bypass Ratio", 
                      "compressor Pressure Ratio","ratio of specific heats",
                      "total temperature at end of combustor",
                      "total temperature at exit of the afterburner",
                      "the different efficiences",
                      "Heating value of the fuel", "Mass flow through inlet"]
            inputVal = []
            for item in inputTerms:
                print("Input the value of " + item)
                if item != "the different efficiences":
                    inputVal.append(float(input()))
                else:
                    effTerms = ["Compressor: ", "Burner: ",
                                "Turbine: ", "Afterburner: ",
                                "Nozzle: "]
                    effVal = []
                    for jtem in effTerms:
                        print("  " + jtem)
                        eff = input()
                        print(eff)
                        effVal.append(eff)
                    inputVal.append(effVal)
            print(inputVal)
            self.setValues(inputVal)
        self.alt = altitude
        self.q = q
        self.BPR = BPR
        self.CPR = CPR
        self.gamma = gamma
        self.Cp = Cp
        self.Tt4 = Tt4
        self.Tt7 = Tt7
        self.nc = nc
        self.nb = nb
        self.nt = nt
        self.nab = nab
        self.nn = nn
        self.Hb = Hb
        self.mDot = mDot
        self.R = 287 #[J/kg K]
        self.Rus = 1717 #[ft lbf/slug °R]
    def changeToSI(self):
        self.alt = self.alt * 0.3048
        self.q = 1500 * 47.880258888889 #Pa
        self.Cp = 0.3 * 4186.
        self.Tt4 = ((self.Tt4 - 32) * (5/9)) + 273.15
        self.Tt7 = ((self.Tt7 - 32) * (5/9)) + 273.15
        Hb = 18500 * 1055.06 * (1/0.453592)
        mDot = 500 * 0.453592 #kg/s
    def setValues(self,inputVal):
        return null
    def runAnalysis(self,hardCode=0,afterburner=1):
        #Atmospheric Conditions
        atmos = atmosphere(self.alt)
        atmos.hCalc()
        self.P0 = atmos.P
        #Stage 0 and Inlet (stage 1)
        self.rho0 = atmos.rho
        self.M0 = np.sqrt((self.q * 2)/(self.gamma * self.P0))
        if hardCode == 1:
            self.M0 = 1
            self.P0 = 101325 #Pa
            self.T0 = 288.15 #K
        if hardCode == 2:
            self.M0 = 4
            self.P0 = (2*self.q)/(self.gamma * self.M0**2)
            atmos = atmosphere(self.P0,1)
            atmos.PCalc()
            self.T0 = atmos.T
            self.h = atmos.h
        self.Tt0 = totalTemp(self.T0,self.M0,self.gamma)
        if self.M0 < 1:
            Pt2Pt0 = 1
        elif self.M0 < 5:
            Pt2Pt0 = 1 - 0.075*(self.M0 - 1)**(1.35)
        else:
            Pt2Pt0 = (800/(self.M0**4 + 935))
        self.Pt0 = self.P0 + self.q
        #Stage 2
        self.Pt2 = self.Pt0 * Pt2Pt0
        del(Pt2Pt0)
        self.Tt2 = self.Tt0
        #Compressor
        self.Tt3 = self.Tt2 * (1 + 
                              (1/self.nc) *
                              (self.CPR**((self.gamma-1)/self.gamma)-1))
        self.Pt3 = self.Pt2 * self.CPR
        self.mDotc = self.mDot/(self.BPR + 1)
        self.mDotb = self.mDot - self.mDotc
        #Burner
        self.f = ((self.Tt4-self.Tt3)/self.Hb)*self.Cp
        self.mDotf = self.mDotc * self.f
        self.Pt4 = self.Pt3
        #Turbine
        self.Tt5 = self.Tt4 - ((self.mDotc * self.Cp * (self.Tt3-self.Tt2))/
                               ((self.mDotc + self.mDotf) * self.Cp)
                              )
        self.Pt5 = (self.Pt4 * (1 - 
                               ((self.Pt3/self.Pt2)**
                               ((self.gamma-1)/self.gamma) - 1)/
                               (self.nt*self.nc *
                               (self.Tt4/self.Tt2))
                               )**(self.gamma/(self.gamma-1))
                   )
        #Mixer
        self.mDotMix = self.mDotc + self.mDotf + self.mDotb
        self.Tt6 = ((self.mDotb/self.mDotMix)*self.Tt2
                    +((self.mDotc + self.mDotf)/self.mDotMix)*self.Tt5)
        self.Pt6 = ((self.mDotb/self.mDotMix)*self.Pt2
                   + ((self.mDotc + self.mDotf)/self.mDotMix)*self.Pt5)
        if afterburner == 1:
            #Afterburner
            self.Pt7 = self.Pt6
            self.fab = ((1 - self.Tt5)/
                  ((self.Hb / self.Cp)-self.Tt7))
            self.mDotfab = self.mDotMix * self.fab
            #Nozzle
            self.P9 = self.P0
            self.Pt9 = self.Pt7
            self.Tt9 = self.Tt7
        else:
            self.P9 = self.P0
            self.Pt9 = self.Pt6
            self.Tt9 = self.Tt6
            self.mDotfab = 0
        self.V9 = np.sqrt(2 * self.Cp * self.Tt9 * self.nn *
                          (1 - 
                           (self.P9/self.Pt9)**((self.gamma-1)/self.gamma))
                         )
        #Final values
        self.spcThrust = ((self.mDotMix + self.mDotfab)*self.V9 -
                          self.mDot * (self.M0 * np.sqrt(self.gamma * self.R * self.T0))
                         )
        self.fuelFlow = self.mDotf + self.mDotfab
        self.TSFC = ((self.fuelFlow/self.mDot)/self.spcThrust)
        self.F = self.spcThrust*(self.mDotMix+self.mDotfab)
        
        
engine.help()

Engine Class Help File:
The engine class takes as input several parameters pertaining to a 
gas turbine engine and calculates information from those starting parameters.
The required inputs are in the following order:
1. Altitude
2. dynamic pressure
3. Flow Bypass Ratio 
4. Compressor Pressure Ratio 
5. ratio of specific heats
6. Total temperature at end of combustor
7. Total temperature at exit of the afterburner
8. A 5 element list containing the efficiences (c,b,t,ab,n)
9. Heating value of the fuel
10. Mass flow through the inlet

These values are set in the class with default values for this problem.
Alternatively, one can run with 'manualEntry = True' to be able to enter
these values manually when the object is created.
Running the 'runAnalysis()' command will generate all of the desired values. 
These include arrays of temperatures,pressures, mach, areas, and values for the
mass flows throughout. There will also be values calculated for the specific thrust
and fuel consumption.



### iii)
Determining thrust and TSFC for $M = 1.0$, $BPR = 0$, and $CPR = 20$. Also with $M_4 = 0.4$ we will find the flow area at the entrance of the turbine.

One part of this problem required us to find the molecular mass of air and kerosene. The molecular mass we use for air is $28.97$ kg/kmol. We then approximated kerosene as dodecane and found a molar mass of $171.48$ kg/kmol.

Also, we found that for a $M=1$ condition, the altitude given from the atmospheric model gave a negative values. Thus, we will not use the given $q$ value and instead assume $P_0 = 101.325$ kPa and that $q$ is whatever it needs to make the relation work.

In [170]:
engine3 = engine()
engine3.changeToSI()
engine3.runAnalysis(hardCode=1,afterburner=0)
print(engine3.TSFC*10**6,"* 10^-6")
print(rounds(engine3.F))
R = engine3.Cp*(1-engine3.gamma**(-1))
T5 = statTemp(engine3.Tt5,0.4,engine3.gamma)
P5 = statPres(engine3.Pt5,0.4,engine3.gamma)
V5 = 0.4 * np.sqrt(engine3.gamma * R * T5)
A = (((engine3.mDot+engine3.mDotf) * R * T5)/
     (P5 * V5))
print(A)

1.4188769066787519 * 10^-6
2234085993455.48
65.32842783936991


Thrust: 1768652965697.677 N

TSFC: 1.7922639913692833 * 10^-6

Area at station 5: 65.32842783936991

A = (mdot R static_T)/(static P and V)

3.5287 sq ft

### iv)
Setting BPR $ = 0.5$, $M_b = 0.5$. Constant pressure mixture.

In [173]:
i=0
cprGuess = 2
print("A",A)
while True:
    i +=1
    engine4=engine(CPR = cprGuess,BPR=0.5)
    engine4.changeToSI()
    engine4.runAnalysis(hardCode=2)
    R = engine4.Cp*(1-engine4.gamma**(-1))
    T5 = statTemp(engine4.Tt5,0.4,engine4.gamma)
    P5 = statPres(engine4.Pt5,0.4,engine4.gamma)
    V5 = 0.4 * np.sqrt(engine4.gamma * R * T5)
    Aguess = (((engine4.mDot+engine4.mDotf) * R * T5)/
         (P5 * V5))
    if np.abs(Aguess - A) < .01:
        print("Aguess", Aguess)
        print("CPR: ", cprGuess)
        break
    if i > 100000:
        print("overflow")
        break
    cprGuess += 0.0001
    
print("Thrust: ",engine4.F)
print("TSFC: ",engine4.TSFC)

A 65.32842783936991
Aguess 65.3279915205259
CPR:  3.4227000000030023
Thrust:  7255647338.910174
TSFC:  1.4891344291507402e-06
6750.036497493751


In [174]:
#Finding bypass area
mDotBy = 500/3 * 0.453592 #kg/s
R = 287
Tb = statTemp(engine4.Tt2,0.5,engine4.gamma)
Pb = statPres(engine4.Pt2,0.5,engine4.gamma)
Vb = 0.5 * np.sqrt(engine4.gamma * R * Tb)
Ab = ((mDotBy * R * Tb)/
      (Pb * Vb))
print(Ab)

1.7808424719106464


### v)
Now, we set $M_0 = 2$. 